In [13]:
import pandas as pd
import re
# import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

import tqdm as tqdm
from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/content/faq.csv')
df.shape

(90, 3)

In [9]:
df.head(10)

,question,answer,category
0,What if I need to know my correct CGPA?,CGPA is seen on the grade card. The CGPA count...,grade card
1,I have cleared my backlog but the result sheet...,GPA and CGPA are not mentioned on backlog gra...,grade card
2,"I have cleared my backlog, but the result on E...",Grade card shows first attempt GPA and CGPA o...,grade card
3,What do I do if I have not received my grade c...,Grade cards should be collected from the stude...,grade card
4,My name/Father's Name/Mother's name is incorre...,Name correction on grade cards/PDC needs to b...,Corrections on grade card /PDC
5,How can I get a new copy of corrected grade ca...,Once the required corrections are done in ERP...,Corrections on grade card /PDC
6,From where should I collect the corrected grad...,You have to collect the corrected grade cards/...,Corrections on grade card /PDC
7,In how many days will I get the corrected grad...,Processing of correction on grade cards will t...,Corrections on grade card /PDC
8,Do I need to submit an Exam form for the end t...,"Yes, it is necessary to fill exam form for ev...",Exam Form Issues
9,"I have an internal backlog, do I need to submi...","Yes, it is necessary to fill exam form for in...",Exam Form Issues


In [8]:
df['question'][1]

'I have cleared my backlog but the result sheet does not show GPA and CGPA. Why?'

In [4]:
doc

NameError: ignored

In [5]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [7]:
get_entities(df['question'][1])

['result sheet', 'GPA']

In [19]:
get_entities(df['answer'])

TypeError: ignored

In [17]:
entity_pairs = []

for i in tqdm(df['answer']):
  entity_pairs.append(get_entities(i))

TypeError: ignored

In [20]:
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1", None, pattern) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

In [23]:
df['question'][1]

'I have cleared my backlog but the result sheet does not show GPA and CGPA. Why?'

In [22]:
get_relation(df['question'][1])

'Why'